# 数据读取

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV,cross_validate,KFold,cross_val_predict
from sklearn.cross_decomposition import PLSRegression
# 实用函数
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score, precision_score, f1_score, roc_auc_score,make_scorer

import time
import spectrapepper as spep

from spectra_processing import *

In [2]:
FTIR_train=pd.read_csv('数据集/FTIR_rabbit_train_ave_mean center.csv',index_col=0)
FTIR_test=pd.read_csv('数据集/FTIR_rabbit_test_ave_mean center.csv',index_col=0)
FTIR_wavenum=np.array(FTIR_train.columns.tolist()[7:],dtype='float')
FTIR_datatrain=FTIR_train.iloc[:,7:]

Raman_train=pd.read_csv('数据集/Raman_rabbit_train_ave_mean center.csv',index_col=0)
Raman_test=pd.read_csv('数据集/Raman_rabbit_test_ave_mean center.csv',index_col=0)
Raman_wavenum=np.array(Raman_train.columns.tolist()[7:],dtype='float')
Raman_datatrain=Raman_train.iloc[:,7:]

Combine_train=pd.concat([FTIR_train,Raman_train.iloc[:,7:]],axis=1)
Combine_test=pd.concat([FTIR_test,Raman_test.iloc[:,7:]],axis=1)
Combine_wavenum=np.array(Combine_train.columns.tolist()[7:],dtype='float')
Combine_datatrain=Combine_train.iloc[:,7:]

In [3]:
data_FTIR_train=FTIR_train.copy()
data_FTIR_test=FTIR_test.copy()


data_FTIR_train['label1'].replace(to_replace='Q',value=1,inplace=True)
data_FTIR_train['label1'].replace(to_replace='S',value=0,inplace=True)
data_FTIR_train['label6'].replace(to_replace='Q',value=1,inplace=True)
data_FTIR_train['label6'].replace(to_replace='QS',value=2,inplace=True)
data_FTIR_train['label6'].replace(to_replace='S',value=0,inplace=True)

data_FTIR_test['label1'].replace(to_replace='Q',value=1,inplace=True)
data_FTIR_test['label1'].replace(to_replace='S',value=0,inplace=True)
data_FTIR_test['label6'].replace(to_replace='Q',value=1,inplace=True)
data_FTIR_test['label6'].replace(to_replace='QS',value=2,inplace=True)
data_FTIR_test['label6'].replace(to_replace='S',value=0,inplace=True)

data_Raman_train=Raman_train.copy()
data_Raman_test=Raman_test.copy()


data_Raman_train['label_1'].replace(to_replace='Q',value=1,inplace=True)
data_Raman_train['label_1'].replace(to_replace='S',value=0,inplace=True)
data_Raman_train['label_6'].replace(to_replace='Q',value=1,inplace=True)
data_Raman_train['label_6'].replace(to_replace='QS',value=2,inplace=True)
data_Raman_train['label_6'].replace(to_replace='S',value=0,inplace=True)

data_Raman_test['label_1'].replace(to_replace='Q',value=1,inplace=True)
data_Raman_test['label_1'].replace(to_replace='S',value=0,inplace=True)
data_Raman_test['label_6'].replace(to_replace='Q',value=1,inplace=True)
data_Raman_test['label_6'].replace(to_replace='QS',value=2,inplace=True)
data_Raman_test['label_6'].replace(to_replace='S',value=0,inplace=True)


data_Combine_train=Combine_train.copy()
data_Combine_test=Combine_test.copy()


data_Combine_train['label1'].replace(to_replace='Q',value=1,inplace=True)
data_Combine_train['label1'].replace(to_replace='S',value=0,inplace=True)
data_Combine_train['label6'].replace(to_replace='Q',value=1,inplace=True)
data_Combine_train['label6'].replace(to_replace='QS',value=2,inplace=True)
data_Combine_train['label6'].replace(to_replace='S',value=0,inplace=True)

data_Combine_test['label1'].replace(to_replace='Q',value=1,inplace=True)
data_Combine_test['label1'].replace(to_replace='S',value=0,inplace=True)
data_Combine_test['label6'].replace(to_replace='Q',value=1,inplace=True)
data_Combine_test['label6'].replace(to_replace='QS',value=2,inplace=True)
data_Combine_test['label6'].replace(to_replace='S',value=0,inplace=True)

In [4]:
x_FTIR_train=data_FTIR_train.iloc[:,7:].values
y_FTIR_train_two=data_FTIR_train['label1'].values
y_FTIR_train_three=data_FTIR_train['label6'].values
x_FTIR_test=data_FTIR_test.iloc[:,7:].values
y_FTIR_test_two=data_FTIR_test['label1'].values
y_FTIR_test_three=data_FTIR_test['label6'].values



x_Raman_train=data_Raman_train.iloc[:,7:].values
y_Raman_train_two=data_Raman_train['label_1'].values
y_Raman_train_three=data_Raman_train['label_6'].values
x_Raman_test=data_Raman_test.iloc[:,7:].values
y_Raman_test_two=data_Raman_test['label_1'].values
y_Raman_test_three=data_Raman_test['label_6'].values


x_Combine_train=data_Combine_train.iloc[:,7:].values
y_Combine_train_two=data_Combine_train['label1'].values
y_Combine_train_three=data_Combine_train['label6'].values
x_Combine_test=data_Combine_test.iloc[:,7:].values
y_Combine_test_two=data_Combine_test['label1'].values
y_Combine_test_three=data_Combine_test['label6'].values

label_1=Combine_test['label1']
label_6=Combine_test['label6']
label_2=Combine_test['label2']

label_two= list(set(label_1))
label_two.sort()
label_three=list(set(label_6))
label_three.sort()
label_post=list(set(label_2))
label_post.sort()

# FTIR建模

In [5]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np 
import time

In [6]:
#打包成函数供后续使用
#评估指标RMSE
def ACC(cvresult,key):
    return (cvresult[key].mean())

#计算参数空间大小
def count_space(param):
    no_option = 1
    for i in param:
        no_option *= len(param[i])
    print(no_option)
    
#在最优参数上进行重新建模验证结果
def rebuild_on_best_param(ad_reg,X,y):
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    result_post_adjusted = cross_validate(ad_reg,X,y,cv=cv,scoring="accuracy"
                                          ,return_train_score=True
                                          ,verbose=True
                                          ,n_jobs=-1)
    print("训练accuracy:{:.3f}".format(ACC(result_post_adjusted,"train_score")))
    print("测试accuracy:{:.3f}".format(ACC(result_post_adjusted,"test_score")))
    

def result_df(model, X_train, y_train, X_test, y_test, metrics=
              [accuracy_score, recall_score, precision_score, f1_score]):
    res_train = []
    res_test = []
    col_name = []
    for fun in metrics:
        res_train.append(fun(y_train,model.predict(X_train)))
        res_test.append(fun(y_test,model.predict(X_test))) 
        col_name.append(fun.__name__)
    idx_name = ['train_eval', 'test_eval']
    res = pd.DataFrame([res_train, res_test], columns=col_name, index=idx_name)
    return res

def result_multi(model, X_train, y_train, X_test, y_test, metrics=
              [accuracy_score, recall_score, precision_score, f1_score]):
    res_train = []
    res_test = []
    col_name = []
    
    res_train.append(accuracy_score(model.predict(X_train), y_train))
    res_test.append(accuracy_score(model.predict(X_test), y_test))
    col_name.append('accuracy_score')
    
    for fun in metrics[1:4]:
        res_train.append(fun(y_train,model.predict(X_train), average='macro'))
        res_test.append(fun(y_test,model.predict(X_test),average='macro')) 
        col_name.append(fun.__name__)
    idx_name = ['train_eval', 'test_eval']
    res = pd.DataFrame([res_train, res_test], columns=col_name, index=idx_name)
    return res

In [7]:
## TPE搜索
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss


def hyperopt_objective(params):
    
    #定义评估器
    #需要搜索的参数需要从输入的字典中索引出来
    #不需要搜索的参数，可以是设置好的某个值
    #在需要整数的参数前调整参数类型
    clf=SVC(cache_size=5000
           ,kernel=params['kernel']
           ,gamma=params['gamma']
           ,degree=int(params['degree'])
           ,coef0=params['coef0']
           ,random_state=225)
    
    cv=KFold(n_splits=10,shuffle=True,random_state=int(params['random_state']))
    result_bench = cross_validate(clf,x_FTIR_train,y_FTIR_train_two,cv=cv,scoring="accuracy"
                          ,return_train_score=True
                          ,verbose=0
                          ,n_jobs=48
                          ,error_score='raise')
 
    
    
    #最终输出结果，由于只能取最小值，所以必须对（-RMSE）求绝对值
    #以求解最小RMSE所对应的参数组合
    return -(np.mean(result_bench['test_score']))
 
    

param_grid_tpe = {'kernel': hp.choice("kernel",['linear','poly','rbf','sigmoid'])
                     ,'gamma': hp.uniform("gamma",0,20)
                     ,'degree':hp.quniform('degree',1,4,1)
                     ,'coef0':hp.uniform("coef0",1,30)
                     ,'random_state':hp.quniform('random_state',1,1000,1)
                    }
# best params:  {'coef0': 9.603275658203557, 'degree': 2.0, 'gamma': 6.119016770591143, 'kernel': 1} 0.776

def param_hyperopt(max_evals=100,n=50):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(n)
    
    #定义代理模型
    #algo = partial(tpe.suggest, n_startup_jobs=20, n_EI_candidates=50)
    params_best = fmin(hyperopt_objective #目标函数
                       , space = param_grid_tpe #参数空间
                       , algo = tpe.suggest #代理模型你要哪个呢？
                       #, algo = algo
                       , max_evals = max_evals #允许的迭代次数
                       , verbose=1
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

## params_best, trials = param_hyperopt(1000,200)

In [8]:
clf=SVC(cache_size=5000
           ,kernel='poly'
           ,gamma=19
           ,degree=2
           ,coef0=16
           ,probability=True
           ,random_state=225)
    
cv=KFold(n_splits=10,shuffle=True,random_state=3763)
result_bench = cross_validate(clf,x_FTIR_train,y_FTIR_train_two,cv=cv,scoring="accuracy"
                          ,return_train_score=True
                          ,verbose=1
                          ,n_jobs=-1)

print('train_score:',np.mean(result_bench['train_score'])
      ,'\n''test_score:',np.mean(result_bench['test_score']))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   11.1s remaining:    7.4s


train_score: 0.9804908967717963 
test_score: 0.7773618538324422


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   14.7s finished


In [9]:
result_df(clf.fit(x_FTIR_train,y_FTIR_train_two),x_FTIR_train,y_FTIR_train_two,x_FTIR_test,y_FTIR_test_two)

,accuracy_score,recall_score,precision_score,f1_score
train_eval,0.979167,0.982143,0.976331,0.979228
test_eval,0.785714,0.761905,0.800000,0.780488


In [10]:
clf.dual_coef_.shape

(1, 147)

# Raman 建模

In [11]:
clf=SVC(cache_size=5000
           ,kernel='poly'
           ,gamma=2.196
           ,degree=2
           ,coef0=24.699
           ,probability=True
           ,random_state=225)
    
cv=KFold(n_splits=10,shuffle=True,random_state=5070)
result_bench = cross_validate(clf,x_Raman_train,y_Raman_train_two,cv=cv,scoring="accuracy"
                          ,return_train_score=True
                          ,verbose=1
                          
                          ,n_jobs=48
                          ,error_score='raise')

print('train_score:',np.mean(result_bench['train_score'])
      ,'\n''test_score:',np.mean(result_bench['test_score']))


[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done   3 out of  10 | elapsed:   26.3s remaining:  1.0min


train_score: 1.0 
test_score: 0.8064171122994652


[Parallel(n_jobs=48)]: Done  10 out of  10 | elapsed:   30.0s finished


In [12]:
result_df(clf.fit(x_Raman_train,y_Raman_train_two),x_Raman_train,y_Raman_train_two,x_Raman_test,y_Raman_test_two)

,accuracy_score,recall_score,precision_score,f1_score
train_eval,1.000000,1.000000,1.000000,1.000000
test_eval,0.833333,0.857143,0.818182,0.837209


# Combine

In [13]:
clf=SVC(cache_size=5000
           ,kernel='poly'
           ,gamma=7.3587
           ,degree=2
           ,coef0=23.179
           ,probability=True
           ,random_state=360)
    
cv=KFold(n_splits=10,shuffle=True,random_state=360)
result_bench = cross_validate(clf,x_Combine_train,y_Combine_train_two,cv=cv,scoring="accuracy"
                          ,return_train_score=True
                          ,verbose=1
                          ,n_jobs=-1
                          ,error_score='raise')

print('train_score:',np.mean(result_bench['train_score'])
      ,'\n''test_score:',np.mean(result_bench['test_score']))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   15.9s remaining:   10.6s


train_score: 1.0 
test_score: 0.8513368983957219


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.9s finished


In [14]:
result_df(clf.fit(x_Combine_train,y_Combine_train_two),x_Combine_train,y_Combine_train_two,x_Combine_test,y_Combine_test_two)

,accuracy_score,recall_score,precision_score,f1_score
train_eval,1.000000,1.000000,1.000000,1.000000
test_eval,0.857143,0.857143,0.857143,0.857143
